In [3]:
import matplotlib.pyplot as plt
import numpy as np
from struct import unpack
from sklearn import cluster
import datetime
import hdbscan
import seaborn as sns
from sklearn.preprocessing import PowerTransformer, normalize, MinMaxScaler, StandardScaler
from tsnecuda import TSNE
from struct import pack
from sklearn_extra.cluster import KMedoids

In [7]:
def transform(_in) :
    #vg / h 분리
    np_vgh = _in
    np_vg = np.delete(_in, 2, axis = 1) #3열 삭제
    np_h = (_in[:,2]).reshape(-1,1) #3열 추출
    #log(h)
    np_logh = np.log(np_h[::]+1)
    #logh 병합
    np_vgh_logh = np.hstack((np_vgh,np_logh))
    # normalized v,g,h,logh
    np_normal_l1_vgh_logh = normalize(np_vgh_logh, axis=0, norm='l1')
    np_normal_l2_vgh_logh = normalize(np_vgh_logh, axis=0, norm='l2')
    np_normal_max_vgh_logh = normalize(np_vgh_logh, axis=0, norm='max')
    #tr standardization
    std_scaler = StandardScaler()
    fitted = std_scaler.fit(np_vgh_logh)
    np_std_vgh_logh = std_scaler.transform(np_vgh_logh)
    #print(np_std_vgh_logh)
    #tr min-max scale
    min_max_scaler = MinMaxScaler()
    min_max_scaler.fit(np_vgh_logh)
    np_min_max_vgh_logh=min_max_scaler.transform(np_vgh_logh)
    #print(np_min_max_vgh_logh)
    #yeo-johnson
    pt_vgh_logh = PowerTransformer(method='yeo-johnson')
    pt_vgh_logh.fit(np_vgh_logh)
    np_yeojohnson_vgh_logh = pt_vgh_logh.transform(np_vgh_logh)

    ret = []
    ret.append(  _in                                  )  
    ret.append(  np.delete(np_vgh_logh, 2, axis = 1)                   )
    ret.append(  np.hstack((np_normal_l1_vgh_logh[:,[0,1]], np_h))     )
    ret.append(  np.hstack((np_normal_l1_vgh_logh[:,[0,1]], np_logh))  ) 
    ret.append(  np.delete(np_normal_l1_vgh_logh, 2, axis = 1)         )
    ret.append(  np.delete(np_normal_l1_vgh_logh, 3, axis = 1)         )
    ret.append(  np.hstack((np_normal_l2_vgh_logh[:,[0,1]], np_h))     )
    ret.append(  np.hstack((np_normal_l2_vgh_logh[:,[0,1]], np_logh))  )
    ret.append(  np.delete(np_normal_l2_vgh_logh, 2, axis = 1)         )  
    ret.append(  np.delete(np_normal_l2_vgh_logh, 3, axis = 1)         )
    ret.append(  np.hstack((np_normal_max_vgh_logh[:,[0,1]], np_h))    )     
    ret.append(  np.hstack((np_normal_max_vgh_logh[:,[0,1]], np_logh)) )    
    ret.append(  np.delete(np_normal_max_vgh_logh, 2, axis = 1)        )
    ret.append(  np.delete(np_normal_max_vgh_logh, 3, axis = 1)        ) 
    ret.append(  np.hstack((np_std_vgh_logh[:,[0,1]], np_h))           )
    ret.append(  np.hstack((np_std_vgh_logh[:,[0,1]], np_logh))        )
    ret.append(  np.delete(np_std_vgh_logh, 2, axis = 1)               )   
    ret.append(  np.delete(np_std_vgh_logh, 3, axis = 1)               )
    ret.append(  np.hstack((np_min_max_vgh_logh[:,[0,1]], np_h))       )     
    ret.append(  np.hstack((np_min_max_vgh_logh[:,[0,1]], np_logh))    )     
    ret.append(  np.delete(np_min_max_vgh_logh, 2, axis = 1)           )     
    ret.append(  np.delete(np_min_max_vgh_logh, 3, axis = 1)           )       
    ret.append(  np.hstack((np_yeojohnson_vgh_logh[:,[0,1]], np_h))    )         
    ret.append(  np.hstack((np_yeojohnson_vgh_logh[:,[0,1]], np_logh)) )          
    ret.append(  np.delete(np_yeojohnson_vgh_logh, 2, axis = 1)        )         
    ret.append(  np.delete(np_yeojohnson_vgh_logh, 3, axis = 1)        )      
    
    return ret


def _TSNE(learning_rate, data) :
    model = TSNE(learning_rate=learning_rate)
    print("TSNE calc : ", end='')
    startTime = datetime.datetime.now()
    transformed = model.fit_transform(data)
    endTime = datetime.datetime.now()
    diffTime = endTime-startTime
    exeTime = diffTime.total_seconds() * 1000
    print(exeTime,'ms')
    return transformed

In [8]:
index = ["vgh", "vglogh", 
         "n_l1_vg_h", "n_l1_vg_logh", "n_l1_vgh", "n_l1_vglogh", 
         "n_l2_vg_h", "n_l2_vg_logh", "n_l2_vgh", "n_l2_vglogh", 
         "n_max_vg_h", "n_max_vg_logh", "n_max_vgh", "n_max_vglogh", 
         "std_vg_h", "std_vg_logh", "std_vgh", "std_vglogh", 
         "mm_vg_h", "mm_vg_logh", "mm_vgh", "mm_vglogh", 
         "pt_vg_h", "pt_vg_logh", "pt_vgh", "pt_vglogh"
        ]

w=256
h=256
size = w*h

all_val_grad_hist = []
sparse_val_grad_hist = []
#'VisMale_128x256x256','bonsai256X256X256B', 'Carp_256x256x512','XMasTree-LO_256x249x256' 
for dataset in ['abdomen_256x256x128'] :
    with open('../volumeCache/%s.raw.2DHistogram.TextureCache'%(dataset), 'rb') as fp:
        Histogram2DYMax = unpack('<f', fp.read(4))[0] #Max of Grad_mag
        for i in range(h) :
            for j in range(w):
                readdata = unpack('<L', fp.read(4))[0]
                all_val_grad_hist.append([i, j, readdata])
                if readdata>=1 and i!=0:
                    sparse_val_grad_hist.append([i, j, readdata])

        np_all_val_grad_hist = np.array(all_val_grad_hist)
        np_sparse_val_grad_hist = np.array(sparse_val_grad_hist)

        ret_all_array = transform(np_all_val_grad_hist)
        ret_sparse_array = transform(np_sparse_val_grad_hist)
        
        tsne_all_array = []
        tsne_sparse_array = []
        
        ###########################################################################################################
        
        #for i in range(len(ret_all_array)):
        #    print("%d_%s_%s"% (i, dataset, index[i]), end='\t')
        #    tsne_all_array.append(_TSNE(100,ret_all_array[i]))
        
        #for i in range(len(tsne_all_array)):
        #    save_tsne_result( tsne_all_array[i],  "tsneCache", "%d_%s_%s"% (i, dataset, index[i]) )
        
        #for i in range(len(tsne_all_array)):
        #    for _k in [10,15,20]:
        #        kmeans(_k,  tsne_all_array[i], np_all_val_grad_hist, 256,256,  "%d_%s_%s"% (i, dataset, index[i]) )
        #    
        #    for _eps in [0.5, 1.0, 1.5, 3.0]:
        #        dbscan(_eps, tsne_all_array[i], np_all_val_grad_hist, 256,256,  "%d_%s_%s"% (i, dataset, index[i]))
        #    Hdbscan(300, 20, 1.0, tsne_all_array[i], np_all_val_grad_hist, 256,256,  "%d_%s_%s"% (i, dataset, index[i])  ) # *
            
        ###########################################################################################################
        
        for i in range(len(ret_sparse_array)):
            print("%s_%d_sparse_%s"% ( dataset, i,index[i]), end='\t')
            tsne_sparse_array.append(_TSNE(100,ret_sparse_array[i]))
            
np_tsne_sparse_array = np.array(tsne_sparse_array)
print(tsne_sparse_array)
            

abdomen_256x256x128_0_sparse_vgh	TSNE calc : 2507.176 ms
abdomen_256x256x128_1_sparse_vglogh	TSNE calc : 751.013 ms
abdomen_256x256x128_2_sparse_n_l1_vg_h	TSNE calc : 920.562 ms
abdomen_256x256x128_3_sparse_n_l1_vg_logh	TSNE calc : 919.006 ms
abdomen_256x256x128_4_sparse_n_l1_vgh	TSNE calc : 739.913 ms
abdomen_256x256x128_5_sparse_n_l1_vglogh	TSNE calc : 722.429 ms
abdomen_256x256x128_6_sparse_n_l2_vg_h	TSNE calc : 742.0949999999999 ms
abdomen_256x256x128_7_sparse_n_l2_vg_logh	TSNE calc : 740.8090000000001 ms
abdomen_256x256x128_8_sparse_n_l2_vgh	TSNE calc : 776.139 ms
abdomen_256x256x128_9_sparse_n_l2_vglogh	TSNE calc : 709.9140000000001 ms
abdomen_256x256x128_10_sparse_n_max_vg_h	TSNE calc : 734.274 ms
abdomen_256x256x128_11_sparse_n_max_vg_logh	TSNE calc : 739.37 ms
abdomen_256x256x128_12_sparse_n_max_vgh	TSNE calc : 728.711 ms
abdomen_256x256x128_13_sparse_n_max_vglogh	TSNE calc : 712.611 ms
abdomen_256x256x128_14_sparse_std_vg_h	TSNE calc : 735.746 ms
abdomen_256x256x128_15_sparse